In [ ]:
# This has to go in its own cell or it screws up the defaults we'll set later
%matplotlib inline

In [ ]:
import numpy as np
import musictoys
import musictoys.audiofile
import musictoys.analysis
import musictoys.spectral
from scrapbook import plot
import librosa
import librosa.feature
import librosa.feature.spectral

In [ ]:
filedata, filerate = musictoys.audiofile.read("audio_files/kronfeld-dreamatic.wav")
#filedata, filerate = musictoys.audiofile.read("audio_files/jfb-back_home.wav")
#filedata, filerate = musictoys.audiofile.read("audio_files/liberty_chaps-get_up_get_down.wav")
samples, samplerate = musictoys.analysis.normalize(filedata, filerate)

In [ ]:
frame_size = 1024
step_size = frame_size / 2
step_rate = samplerate / step_size
num_frames = int((len(samples) - frame_size + 1) / step_size)
graph_height = 256
gridx, gridy = np.mgrid[0:num_frames, 0:graph_height]

In [ ]:
hann = np.hanning(frame_size)

spectrogram = []
for i in range(num_frames):
    # get the clip and weight according to the hann window
    clip = hann * samples[i*step_size:i*step_size+frame_size]
    # take the FFT and discard the symmetrical components
    spectrum = np.fft.rfft(clip)#[:frame_size/2]
    # add this spectrum to the series of spectra
    spectrogram.append(spectrum)

spectrogram = np.array(spectrogram)
# take absolute value to get magnitude
spectrogram = np.absolute(spectrogram) / np.sum(hann)
# square to get power
powerspectrogram = np.square(spectrogram)
# convert to decibels, clip noise floor
loudnessgram = (10 * np.log10(powerspectrogram)).clip(-120)
# compute the center frequency of each FFT bin
nyquist_freq = samplerate / 2.0
fft_bin_freq = np.arange(spectrogram.shape[1]) * nyquist_freq / float(spectrogram.shape[1])
half_nyquist = nyquist_freq / 2.0

In [ ]:
plot.gram(loudnessgram, x=step_rate, y=fft_bin_freq, cmap='plasma')

# ======== CENTROIDS

In [ ]:
plot.line(musictoys.spectral.centroid(spectrogram, samplerate), x=step_rate)

In [ ]:
plot.line(librosa.feature.spectral_centroid(S=spectrogram.T, sr=samplerate)[0])

# ======== SPREAD/BANDWIDTH

In [ ]:
plot.line(musictoys.spectral.spread(spectrogram, samplerate), x=step_rate)

In [ ]:
plot.line(librosa.feature.spectral_bandwidth(S=spectrogram.T, sr=samplerate)[0])

# ======== SPREAD SCALED BY CENTROID


In [ ]:
def unholy(spec):
    freqs = np.linspace(0, samplerate / 2.0, spec.shape[-1], endpoint=True)
    peak_bin = np.argmax(spec, axis=-1)
    plot.line(peak_bin)
    peak_freq = freqs[peak_bin]
    peak_mag = np.take(spec[:], peak_bin, axis=0)
    print peak_mag.shape
    plot.line(peak_mag)
    plot.line(peak_freq)
    
unholy(spectrogram)
    
#peak_frequency_per_frame = spectrogram[:,peak_frequency_per_bin]
#print peak_frequency_per_frame
#unholy = musictoys.spectral.spread(spectrogram, samplerate) / 
#plot.line(musictoys.spectral.spread(spectrogram, samplerate), x=step_rate)

# ======== VARIANCE

In [ ]:
plot.line(musictoys.spectral.variance(spectrogram), x=step_rate)


In [ ]:
def pymir_variance(spec):
    return np.var(np.abs(spec), axis=-1)


In [ ]:
plot.line(pymir_variance(spectrogram), x=step_rate)

# ======== CREST

In [ ]:
plot.line(musictoys.spectral.crest(spectrogram), x=step_rate)


# ======== ENTROPY

In [ ]:
plot.line(musictoys.spectral.entropy(spectrogram), x=step_rate)

# ======== FLATNESS

In [ ]:
plot.line(musictoys.spectral.flatness(spectrogram, power=1.5), x=step_rate)

In [ ]:
def librosa_spectral_flatness(S=None, amin=1e-10, power=2.0):
    S_thresh = np.maximum(amin, S ** power)
    gmean = np.exp(np.mean(np.log(S_thresh), axis=0, keepdims=True))
    amean = np.mean(S_thresh, axis=0, keepdims=True)
    return gmean / amean

plot.line(librosa_spectral_flatness(spectrogram.T, power=1.5)[0], x=step_rate)

# ======== ROLLOFF

In [ ]:
plot.line(musictoys.spectral.rolloff(spectrogram, samplerate, fraction=0.85))

In [ ]:
plot.line(librosa.feature.spectral.spectral_rolloff(S=spectrogram.T, sr=samplerate, roll_percent=.85)[0])

In [ ]:
worth investigating?
- skewness
- kurtosis
- slope
- decrease
- high-frequency content